# [German Traffic Sign Recognition Benchmark](https://bit.ly/2RuaUCB)
### Celem jest pokazanie, jak zastosować transfer wiedzy do rozpoznawanie znaków drogowych.

### Przykłady znaków:

![](../images/gts.jpeg)

In [1]:
import pickle

import numpy as np
np.random.seed(2019)
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image

from skimage import color, exposure
from skimage.transform import resize

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from collections import Counter

%matplotlib inline

## Wczytujemy dane

In [2]:
training_file = "../input/traffic_sign/train.p"
test_file     =  "../input/traffic_sign/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)

with open(test_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print(X_train.shape, X_test.shape)

(34799, 32, 32, 3) (4410, 32, 32, 3)


## Przygotowanie danych

In [3]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)    
    
input_shape = X_train.shape[1:]
num_classes = 43

## Architektura

In [4]:
def get_doubled_cnn3_with_dropout():
    return Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(), 
        
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])


model = get_doubled_cnn3_with_dropout()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

2022-11-11 16:05:47.581728: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Trenujemy

In [5]:
history = model.fit(X_train, y_train,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=(X_test, y_test))

2022-11-11 16:05:51.741717: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/3
272/272 [==============================] - 36s 131ms/step - loss: 4.0824 - accuracy: 0.1754 - val_loss: 0.9030 - val_accuracy: 0.7533
Epoch 2/3
272/272 [==============================] - 36s 131ms/step - loss: 0.7288 - accuracy: 0.7861 - val_loss: 0.2149 - val_accuracy: 0.9454
Epoch 3/3
272/272 [==============================] - 36s 132ms/step - loss: 0.2710 - accuracy: 0.9207 - val_loss: 0.0773 - val_accuracy: 0.9830


## Sprawdzamy

In [6]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("CNN Error: %.2f%%" % (100-score[1]*100))

Test loss: 0.07734020054340363
Test accuracy: 0.9829931855201721
CNN Error: 1.70%


## VGG

Przygotowujemy dane:

In [7]:
X_train_48 = []
X_test_48 = []

for i in tqdm(range(X_train.shape[0])):
    resized_img = resize(X_train[i], output_shape=(48, 48), mode='reflect')
    X_train_48.append(resized_img)
    
X_train_48 = np.array(X_train_48)
    
for i in tqdm(range(X_test.shape[0])):
    resized_img = resize(X_test[i], output_shape=(48, 48), mode='reflect')
    X_test_48.append(resized_img)
    
X_test_48 = np.array(X_test_48)

100%|███████████████████████████████████████████████| 4410/4410 [00:02<00:00, 2015.36it/s]


Próba #1 - zamrażamy wszystkie warstwy konwolucyjne.

In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [9]:
model = Sequential([
    base_model,
    
    Flatten(), 
        
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 43)                5547      
Total params: 14,785,899
Trainable params: 71,211
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
history = model.fit(X_train_48, y_train,
          batch_size=1024,
          epochs=3,
          verbose=1,
          validation_data=(X_test_48, y_test))

Epoch 1/3
34/34 [==============================] - 140s 4s/step - loss: 3.4659 - accuracy: 0.1256 - val_loss: 2.7796 - val_accuracy: 0.2365
Epoch 2/3
34/34 [==============================] - 139s 4s/step - loss: 2.4977 - accuracy: 0.2983 - val_loss: 2.3473 - val_accuracy: 0.3229
Epoch 3/3
34/34 [==============================] - 142s 4s/step - loss: 2.0977 - accuracy: 0.3920 - val_loss: 2.1617 - val_accuracy: 0.3612


## Przydatne linki:
1. [Pipelines, Mind Maps and Convolutional Neural Networks](https://bit.ly/3esR43N)
2. [ResNet for Traffic Sign Classification With PyTorch](https://bit.ly/2RvifSq)
3. [How to use transfer learning and fine-tuning in Keras and Tensorflow to build an image recognition system and classify (almost) any object](https://bit.ly/3bdvpKW)